# Crypto-Dashboard Project


In [2]:
# General Imports
from binance.client import Client
import pandas as pd
import pandas_ta as ta
import datetime as dt
from datetime import datetime
from time import time
import plotly.express as px
import plotly.graph_objects as go 
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
from dotenv import load_dotenv
import os

In [3]:
# API Details
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

load_dotenv()

client = Client(api_key, api_secret)

In [4]:
# excluded untradeable tokens
err_tokens = ['BULL', 'BEAR', 'DOWN', 'UP']

# get all binance supported pairs
symbols_list = list(info['symbol']
                    for info in client.get_exchange_info()["symbols"] if (info['symbol'].endswith("USDT") and all(lev not in info['symbol'] for lev in err_tokens)))

sym_length = len(symbols_list)

# get 200 days worth of data (daily and weekly)
d_start = (dt.datetime.now() - dt.timedelta(days=200)).strftime("%m-%d-%Y")
d_end = dt.datetime.now().strftime("%m-%d-%Y")

w_start = (dt.datetime.now() - dt.timedelta(days=7)).strftime("%m-%d-%Y")
w_end = dt.datetime.now().strftime("%m-%d-%Y")

# master_df holds 30 days worth of data and technical indicators
# quant_df holds respective changes

master_df = pd.DataFrame([])
quant_df = pd.DataFrame([],columns=["Symbol","1D Δ","2D Δ","7D Δ","1W Δ"])
sentiment = {
    'bullish': 0,
    'bearish': 0
}

test_df = pd.DataFrame([])

In [5]:
err_symbols = []
for symbol in symbols_list:
    
    # binance api call to get ohlcv values
    d_klines = [[float(num) for num in dta[:6]] for dta in client.get_historical_klines(
        symbol, Client.KLINE_INTERVAL_1DAY, d_start, d_end)]

    # convert epoch time to readable time
    for x in d_klines:
        x[0] = datetime.fromtimestamp(int(x[0]/1000)).strftime("%m-%d-%Y")
    

    # initialize dataframe and calculate needed indicator values
    df = pd.DataFrame(
        d_klines, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['Symbol'] = symbol

    # some pairs don't have enough data
    try:
        rsi = round(df.ta.rsi(length=14), 3)
        chop = round(df.ta.chop(length=28),3)
        stoch = df.ta.stoch()
        ma_20 = round(df.ta.sma(length=20), 4)
        ma_50 = round(df.ta.sma(length=50), 4)
        ma_100 = round(df.ta.sma(length=100), 4)
    except Exception as e:
        print(symbol, e)
        continue

    # assign calculated values

    close = df['close']
     
    df['RSI(14)'] = rsi
    df['CHOP(28)'] = chop
    df['STOCH_K'] = round(stoch['STOCHk_14_3_3'],2)
    df['STOCH_D'] = round(stoch['STOCHd_14_3_3'],2)
    df['MA(20)'] = ma_20
    df['MA(50)'] = ma_50
    df['MA(100)'] = ma_100

    # boolean series
    try:
        bluesky = (close > ma_20) & (ma_20 > ma_50) & (ma_50 > ma_100)
        retrace = (close < ma_20) & (ma_20 > ma_50) & (ma_50 > ma_100)
        reset = (ma_20 < ma_50) & (ma_50 > ma_100)
        reversal = (ma_20 > ma_50) & (ma_50 < ma_100)
        downtrend = (ma_100 > ma_50) & (ma_50 > ma_20)
        df['bluesky'] = bluesky
        df['retrace'] = retrace
        df['reset'] = reset
        df['reversal'] = reversal
        df['downtrend'] = downtrend
    except Exception as e:
        print(symbol,e)
        continue


    # assign calculated values
   

    # get the right week and weekly data
    w_start = (dt.datetime.now() - dt.timedelta(days=14)).strftime("%m-%d-%Y")
    w_end = dt.datetime.now().strftime("%m-%d-%Y")

    w_klines = [[float(num) for num in dta[:6]] for dta in client.get_historical_klines(
        symbol, Client.KLINE_INTERVAL_1WEEK, w_start, w_end)]

    # convert to readable date
    for week in w_klines:
        week[0] = datetime.fromtimestamp(int(week[0]/1000)).strftime("%m-%d-%Y")

    wdf = pd.DataFrame(w_klines,columns=['date','open','high','low','close','volume'])

    # calculate changes
    try:
        D1chg = (df.iloc[-1].close - df.iloc[-2].open) / df.iloc[-2].open
        D2chg = (df.iloc[-1].close - df.iloc[-3].open) / df.iloc[-3].open
        D7chg = (df.iloc[-1].close - df.iloc[-8].open) / df.iloc[-8].open

        weeklydf = pd.DataFrame(w_klines,columns=['date','open','high','low','close','volume'])
        # make sure to get last week and not this week (e.g. during Mondays)
        lastweek = wdf[wdf['date'] < w_end].iloc[-1]
        # whichever is higher weekly open/close
        weekly_sig_level = lastweek.close if lastweek.close > lastweek.open else lastweek.open
        Wchg = 0 if weeklydf.empty else (df.iloc[-1].close - weekly_sig_level) / weekly_sig_level

        quant_row = pd.DataFrame([[symbol,D1chg,D2chg,D7chg,Wchg,df.iloc[-1]['RSI(14)']]],columns=["Symbol","1D Δ","2D Δ","7D Δ","1W Δ","rsi"])
        quant_df = pd.concat([quant_df, quant_row],ignore_index=True)
    except Exception as e:
        err_symbols.append(symbol)

    # determine bias

    bias = "bullish" if df.iloc[-1]['RSI(14)'] > df.iloc[-1]['CHOP(28)'] else "bearish"
    sentiment[bias] += 1


    # only get last 30 trading days for ToT
    master_df = master_df.append(df.iloc[len(df.index)-30:])

    # master_df = master_df.set_index("Symbol")
    
    


BCCUSDT 'AnalysisIndicators' object has no attribute '_df'
VENUSDT 'AnalysisIndicators' object has no attribute '_df'
BCHABCUSDT 'AnalysisIndicators' object has no attribute '_df'
BCHSVUSDT 'AnalysisIndicators' object has no attribute '_df'
USDSUSDT 'AnalysisIndicators' object has no attribute '_df'
USDSBUSDT 'AnalysisIndicators' object has no attribute '_df'
ERDUSDT 'AnalysisIndicators' object has no attribute '_df'
STORMUSDT 'AnalysisIndicators' object has no attribute '_df'
HCUSDT 'AnalysisIndicators' object has no attribute '_df'
MCOUSDT 'AnalysisIndicators' object has no attribute '_df'
STRATUSDT 'AnalysisIndicators' object has no attribute '_df'
XZCUSDT 'AnalysisIndicators' object has no attribute '_df'
LENDUSDT 'AnalysisIndicators' object has no attribute '_df'
BKRWUSDT 'AnalysisIndicators' object has no attribute '_df'
DAIUSDT 'AnalysisIndicators' object has no attribute '_df'


In [6]:
# Keep track of trends for Trend of Trends
bluesky = []
retrace = []
reset = []
reversal = []
downtrend = []
lacking_data = []

In [7]:
# Calculate number of trends per day
for date in list(master_df[-30:]['date']):
    current = master_df[master_df['date'] == date]
    
    bluesky.append(current['bluesky'].value_counts().get(True)/sym_length if current['bluesky'].value_counts().get(True) != None else 0)

    retrace.append(current['retrace'].value_counts().get(True)/sym_length if current['retrace'].value_counts().get(True) != None else 0)

    reset.append(current['reset'].value_counts().get(True)/sym_length if current['reset'].value_counts().get(True) != None else 0)

    reversal.append(current['reversal'].value_counts().get(True)/sym_length if current['reversal'].value_counts().get(True) != None else 0)
    
    downtrend.append(current['downtrend'].value_counts().get(True)/sym_length if current['downtrend'].value_counts().get(True) != None else 0)

        

In [8]:
# new function to add when array contains None
def newsum(arr):
    if None in arr:
        arr = list(arr)
        arr = list(filter(lambda x : x!= None,arr))
        return sum(arr)
    else:
        return sum(arr)


In [9]:
ToT_mapper = {
    "bluesky": "#60A5FA",
    "retrace": "#059669",
    "reset": "#F59E0B",
    "reversal": "#92400E",
    "downtrend": "#DC2626"
}


# Line Traces
line_data = []
dates = master_df['date'].unique()
for trend,color in ToT_mapper.items():
    scatter_chart = go.Scatter(
        x=dates,
        y=eval(trend),
        marker_color = color,
        name=trend,
        mode="lines+markers"
    )
    line_data.append(scatter_chart)


In [10]:
# Trend of Trends Line Graph

line_layout = go.Layout(    
    margin=dict(
        l=20,
        r=20, 
        t=30, 
        b=20
    ),
    plot_bgcolor="#FFF",
    xaxis=dict(
        visible=False, 
        showgrid=False
    ),
    yaxis=dict(
        tick0=0, 
        dtick=0.25,
        tickformat=',.0%',
        showgrid=False,
        linecolor="#BCCCDC"
    ),
    height = 280
)


line_fig = go.Figure(
    data = line_data,
    layout = line_layout
)

line_fig



In [11]:
# Trend of Trends Pie Graph
colors = ["#60A5FA","#059669","#F59E0B", "#92400E", "#DC2626"]
lacking = 1-bluesky[-1]-retrace[-1]-reset[-1]-reversal[-1]-downtrend[-1]
trends = ["Uptrend - Blueskies","Uptrend - Retrace", "Sideways - Reset", "Sideways - Reversal","Downtrend", "New / Not Enough Data"]

pie_trends = [bluesky[-1],retrace[-1],reset[-1],reversal[-1],downtrend[-1],lacking]
pie_fig = px.pie(names=trends,values=pie_trends)
pie_fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
pie_fig.update_layout(height=280)
pie_fig.update_traces(hoverinfo='label+percent', textfont_size=16,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

In [12]:
twomean = quant_df['2D Δ'].mean()
twomean

0.0648909259783923

In [13]:
sevenmean = quant_df['7D Δ'].mean()

In [14]:
revised = quant_df[(quant_df['2D Δ'] > twomean) & (quant_df['7D Δ'] > sevenmean)]
revised = revised.fillna(0)
revised.rsi

6      80.696
15     74.126
30     70.453
31     74.678
33     73.985
39     71.442
40     87.484
42     71.307
43     82.004
44     72.387
50     82.045
53     68.631
54     71.774
60     78.423
63     70.514
67     67.616
72     80.343
73     83.159
75     77.164
78     69.884
92     83.214
94     71.190
99     64.798
106    69.552
122    67.281
124    75.549
131    85.068
135    73.543
138    78.599
139    72.683
144    77.773
145    69.237
147    67.503
148    71.887
154    95.686
155    74.966
156    78.327
160    66.220
162    79.569
164    85.525
166    72.874
167    67.585
168    77.729
169    75.376
182    75.452
191    87.538
202    59.022
203    64.528
206    66.920
209    69.230
210    69.433
213    67.774
217    70.384
229    70.655
233    70.880
239    81.465
Name: rsi, dtype: float64

In [15]:
# Quants Scatter Plot
scatter_fig = px.scatter(revised,x='2D Δ',y='7D Δ',color="rsi",color_continuous_scale="Rainbow")
scatter_fig.update_traces(
    mode = "markers+text",
    text = revised['Symbol'],
    textposition = "top left",
    textfont_size=10,
)
scatter_fig.update_layout(
    xaxis_title="2 Day Change",
    yaxis_title="7 Day Change",
    yaxis_visible=True,
    xaxis_visible=True,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    plot_bgcolor="#FFFFFF"
)
scatter_fig.update_yaxes(tick0=0, dtick=0.25,tickformat=',.0%',)
scatter_fig.update_xaxes(tick0=0, dtick=0.10,tickformat=',.0%',)
scatter_fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
scatter_fig.update_layout(height=548)


In [16]:
app = JupyterDash(__name__)

app.layout=html.Div([
    # Left
    html.Div(className="inline-block w-1/12 bg-white",),
    # Middle
    html.Div([
        html.Div([
            # Header
            html.Div([
                html.Div("Dashboard",className="lg-text"),
                html.Div("Updated as of xxxx",className="sm-text")
            ],className="flex flex-col  mb-6"),

            # Trends Boxes
            html.Div([
                html.Div(
                    html.Div([
                        html.Div("Uptrend - Blueskies",className="sm-text"),
                        html.Div("78",className="text-5xl pt-2 pb-2 card-body"),
                        html.Div([
                            html.Div([
                                html.Img(src="./assets/up.svg",className="w-2 h-3")
                            ],className="bg-green-300 w-4 h-4 rounded-full flex items-center justify-center"),
                            html.Div("+",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'}),
                            html.Div("69.12%",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'})
                        ],className="flex space-x-1 items-center flex-row"),
                    ],className="flex flex-col items-center"),className="card bg-white"),

                    
                html.Div(
                    html.Div([
                        html.Div("Uptrend - Retrace",className="sm-text"),
                        html.Div("78",className="text-5xl pt-2 pb-2 card-body"),
                        html.Div([
                            html.Div([
                                html.Img(src="./assets/up.svg",className="w-2 h-3")
                            ],className="bg-green-300 w-4 h-4 rounded-full flex items-center justify-center"),
                            html.Div("+",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'}),
                            html.Div("69.12%",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'})
                        ],className="flex space-x-1 items-center flex-row"),
                    ],className="flex flex-col items-center"),className="card bg-white"),

                html.Div(
                    html.Div([
                        html.Div("Sideways - Reset",className="sm-text"),
                        html.Div("78",className="text-5xl pt-2 pb-2 card-body"),
                        html.Div([
                            html.Div([
                                html.Img(src="./assets/up.svg",className="w-2 h-3")
                            ],className="bg-green-300 w-4 h-4 rounded-full flex items-center justify-center"),
                            html.Div("+",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'}),
                            html.Div("69.12%",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'})
                        ],className="flex space-x-1 items-center flex-row"),
                    ],className="flex flex-col items-center"),className="card bg-white"),
                
                html.Div(
                    html.Div([
                        html.Div("Sideways - Reversal",className="sm-text"),
                        html.Div("78",className="text-5xl pt-2 pb-2 card-body"),
                        html.Div([
                            html.Div([
                                html.Img(src="./assets/up.svg",className="w-2 h-3")
                            ],className="bg-green-300 w-4 h-4 rounded-full flex items-center justify-center"),
                            html.Div("+",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'}),
                            html.Div("69.12%",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'})
                        ],className="flex space-x-1 items-center flex-row"),
                    ],className="flex flex-col items-center"),className="card bg-white"),

                html.Div(
                    html.Div([
                        html.Div("Downtrend",className="sm-text"),
                        html.Div("78",className="text-5xl pt-2 pb-2 card-body"),
                        html.Div([
                            html.Div([
                                html.Img(src="./assets/up.svg",className="w-2 h-3")
                            ],className="bg-green-300 w-4 h-4 rounded-full flex items-center justify-center"),
                            html.Div("+",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'}),
                            html.Div("69.12%",className="text-sm",style={'color':'#1BCA8E','fontFamily': 'Roboto'})
                        ],className="flex space-x-1 items-center flex-row"),
                    ],className="flex flex-col items-center"),className="card bg-white"),
            
            ],className="flex flex-row justify-between mb-5"),
            
            # Scatter Plots
            html.Div([
                
                html.Div([
                    html.Div("Scatter (2d x 7d)",className="md-text"),
                    html.Div([
                        dcc.Graph(
                            id="scatter-2d-vs-7d",
                            figure = scatter_fig,
                        )],className=" flex-grow rounded-lg shadow-md bg-white"),
                ],className=" flex flex-col w-full h-144"),

                

            
            
            ],className="flex flex-row space-x-4 mb-5 "),

            # Trends of Trends and Sentiment
            html.Div([

                # Trend of Trends
                html.Div([
                    html.Div([
                        html.Div("Pie of Trends", className="md-text"),
                        html.Div([
                             dcc.Graph(
                                id="pie-trend-of-trends",
                                figure=pie_fig
                            )],className="flex-grow rounded-lg shadow-md bg-white"),
                    ],className="flex flex-col w-1/2 h-80"),
                    
                    html.Div([
                        html.Div("Sentiment Gauge", className="md-text"),
                        html.Div([
                        
                            daq.Gauge(
                                id="sentiment-gauge",
                                label =" ",
                                color={"gradient":True,"ranges":{"red":[0,50],"yellow":[50,75],"green":[75,100]}},
                                showCurrentValue = True,
                                value=sentiment['bullish']/(sentiment['bullish']+sentiment['bearish'])*100,
                                min=0,
                                max=100,
    
                            )],id="gaugeeee",className="flex-grow rounded-lg shadow-md bg-white pt-5",style={'height':280}),
                            
                    ],className="flex flex-col w-1/2 h-80")

                ],className="flex flex-row w-1/2 space-x-4 h-80 "),

                html.Div([
                    html.Div("Trend of Trends", className="md-text"),
                    html.Div([
                        dcc.Graph(
                        id='line-trend-of-trends',
                        figure=line_fig
                    )],className="flex-grow rounded-lg shadow-md bg-white"),
                ],className="flex flex-col w-1/2 h-80")
               
            
            ],className="flex flex-row space-x-4 h-80"),
        ],className="mt-8 mb-8")
        

    ],id="MIDDLE",className="flex flex-col w-11/12 pl-24 pr-24 bg-frame-gray"),

    # Right
    # html.Div(className="inline-block w-1/12 bg-white"),


],className="flex flex-row h-screen",)

app.run_server(port=6969)

Dash app running on http://127.0.0.1:6969/
